<a href="https://colab.research.google.com/github/TAlkam/-Probability-Stats-for-AI/blob/main/CKD_data_prepration_(6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Preparation and Decision Stump

In [ ]:
from google.colab import files
import pandas as pd
import io

# Upload file
uploaded = files.upload()

Saving ckd-dataset-v2 (2).csv to ckd-dataset-v2 (2) (4).csv


Steps to take:

1. Importing Libraries:
The necessary libraries and modules are imported. These include `numpy`, `pandas`, `train_test_split` from `sklearn.model_selection`, `LogisticRegression` from `sklearn.linear_model`, `confusion_matrix`, `accuracy_score` from `sklearn.metrics`, `SimpleImputer` from `sklearn.impute`, and `OneHotEncoder` from `sklearn.preprocessing`.

2. Defining a Function:
The function `process_column` is defined to handle the data preprocessing step. This function checks if a value in a column is 'discrete' or contains a '-', or is a float. It returns NaN for 'discrete', the average of the two numbers if the value contains a '-', and the float value if it's a float. If none of these conditions are met, it returns NaN.

3. Loading the Dataset:
The dataset is loaded from a CSV file using `pd.read_csv`.

4. Data Preprocessing:
The `process_column` function is applied to the necessary columns of the dataframe. The target column 'class' is converted to integer type, where 'ckd' is represented as 1 and 'notckd' as 0. Missing values in the dataframe are filled with the mean of the respective column. Then, categorical variables are one-hot encoded, and the original categorical columns are dropped from the dataframe.

5. Splitting the Dataset:
The dataset is split into features (X) and target (y). Then, it's further split into training and testing sets using `train_test_split` function.

6. Data Imputation:
A `SimpleImputer` object is created to fill any remaining missing values in the dataset with the mean of the respective column. This imputer is fit on the training data and then used to transform both training and testing data.

7. Training the Model:
A Logistic Regression model is trained using the imputed training data.

8. Making Predictions:
The model is used to make predictions on the test data.

9. Evaluating the Model:
The accuracy of the model is printed out, and a confusion matrix is displayed to evaluate the performance of the model.

Note: This code is quite comprehensive and incorporates several good practices like handling missing values, converting data types, one-hot encoding categorical variables, and splitting the dataset into training and testing sets. It also makes use of logistic regression, a simple and commonly used machine learning algorithm for binary classification problems.

**Hypertension (htn)**

In [ ]:
# Import the necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Load your data into a pandas DataFrame
# df = pd.read_csv('your_data.csv')

# Assume that 'df' is your DataFrame, 'htn' is the feature column, and 'classification' is the target column
# Also assuming that 'classification' is a binary variable

# Handle missing values in 'htn' column by filling with the mode
df['htn'] = df['htn'].fillna(df['htn'].mode()[0])

# Encode 'htn' column to numerical values if it's categorical
le = LabelEncoder()
df['htn'] = le.fit_transform(df['htn'])

X = df[['htn']]  # feature
y = df['class']  # target

# Split the dataset into 70% training data and 30% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Create an instance of DecisionTreeClassifier with max_depth = 1 (Decision Stump)
clf = DecisionTreeClassifier(max_depth=1, random_state=0)

# Train the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Print the accuracy of the model
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 0.7377049180327869


**Diabetes Mellitus (dm)**

In [ ]:
# Import the necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Load your data into a pandas DataFrame
# df = pd.read_csv('your_data.csv')

# Assume that 'df' is your DataFrame, 'dm' is the feature column, and 'classification' is the target column
# Also assuming that 'classification' is a binary variable

# Handle missing values in 'dm' column by filling with the mode
df['dm'] = df['dm'].fillna(df['dm'].mode()[0])

# Encode 'dm' column to numerical values if it's categorical
le = LabelEncoder()
df['dm'] = le.fit_transform(df['dm'])

X = df[['dm']]  # feature
y = df['class']  # target

# Split the dataset into 70% training data and 30% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Create an instance of DecisionTreeClassifier with max_depth = 1 (Decision Stump)
clf = DecisionTreeClassifier(max_depth=1, random_state=0)

# Train the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Print the accuracy of the model
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 0.7049180327868853


**Hemoglobin A1c (hemo)**

In [ ]:
# Necessary imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer

def process_column(col):
    if 'discrete' in str(col):
        return np.nan  # return NaN if 'discrete' is in column
    if '-' in str(col):
        low, high = map(float, str(col).split('-'))  # split on '-', convert to float
        return (low + high) / 2  # return the average
    else:
        try:
            return float(col)  # convert to float
        except ValueError:
            return np.nan  # if conversion to float fails, return NaN

# Load the dataset
df = pd.read_csv('ckd-dataset-v2 (2).csv')

# Apply process_column function to necessary columns
df['hemo'] = df['hemo'].apply(process_column)

# Convert 'class' to integer type
df['class'] = (df['class'] == 'ckd').astype(int)

# Fill missing values with the mean of the respective column
df['hemo'] = df['hemo'].fillna(df['hemo'].mean())

# Split the dataset into features and target
X = df[['hemo']]  # Select only the 'hemo' column as the feature
y = df['class']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use mean imputation
imputer = SimpleImputer(strategy='mean')

# Fit on the training data
imputer.fit(X_train)

# Transform both training and testing data
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

# Train the model using the imputed training data
model = DecisionTreeClassifier(max_depth=1)
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Print out the accuracy and confusion matrix
print(f"Accuracy: {round(accuracy_score(y_test, y_pred), 2)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")



Accuracy: 0.88
Confusion Matrix:
[[13  0]
 [ 5 23]]


# Logistic regression

The Logistic Regression model from the `sklearn.linear_model` module.

Here's a step-by-step breakdown of what the code is doing:

1. `StandardScaler()`: This creates an instance of the StandardScaler class, which will be used to standardize the features by removing the mean and scaling to unit variance. This is often a good preprocessing step for many machine learning algorithms.

2. `scaler.fit_transform(X_train)`: This fits the scaler to the training data and then transforms the training data. "Fitting" the scaler means that it learns the parameters (mean and standard deviation for standardization) of the training data.

3. `scaler.transform(X_test)`: This uses the scaler that was fitted to the training data to transform the test data. It's important to note that the same scaler is used to transform both the training and test data to ensure that they are scaled in the same way.

4. `LogisticRegression(max_iter=1000)`: This creates an instance of the LogisticRegression class. The `max_iter=1000` argument sets the maximum number of iterations for the solver to converge, which can be necessary for larger datasets.

5. `model.fit(X_train, y_train)`: This fits the logistic regression model to the training data. "Fitting" the model means that it learns the relationship between the features (X_train) and the target (y_train).

6. `model.predict(X_test)`: This uses the fitted model to make predictions on the test data.

7. `accuracy_score(y_test, y_pred)`: This calculates the accuracy of the model by comparing the predicted values to the actual values.

So, in summary, this code is using logistic regression to make predictions on the test data and then calculating the accuracy of those predictions.

**Hypertension (htn)**

In [ ]:
# Import the necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Load your data into a pandas DataFrame
# df = pd.read_csv('your_data.csv')

# Assume that 'df' is your DataFrame, 'htn' is the feature column, and 'classification' is the target column
# Also assuming that 'classification' is a binary variable

# Handle missing values in 'htn' column by filling with the mode
df['htn'] = df['htn'].fillna(df['htn'].mode()[0])

# Encode 'htn' column to numerical values if it's categorical
le = LabelEncoder()
df['htn'] = le.fit_transform(df['htn'])

X = df[['htn']]  # feature
y = df['class']  # target

# Split the dataset into 70% training data and 30% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Create an instance of LogisticRegression
lr = LogisticRegression()

# Train the model
lr.fit(X_train, y_train)

# Make predictions
y_pred = lr.predict(X_test)

# Print the accuracy of the model
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 0.7377049180327869


**Hemoglobin A1c (hemo)**

In [ ]:
# Import the necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# Load your data into a pandas DataFrame
# df = pd.read_csv('your_data.csv')

# Assume that 'df' is your DataFrame, 'hemo' is the feature column, and 'classification' is the target column
# Also assuming that 'classification' is a binary variable

# Handle missing values in 'hemo' column
df['hemo'] = df['hemo'].fillna(df['hemo'].mean())

X = df[['hemo']]  # feature
y = df['class']  # target

# Split the dataset into 70% training data and 30% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Create an instance of LogisticRegression
lr = LogisticRegression()

# Train the model
lr.fit(X_train, y_train)

# Make predictions
y_pred = lr.predict(X_test)

# Print the accuracy of the model
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 0.8032786885245902


**Diabetes Mellitus (dm)**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

# Assuming 'dm' is your feature and 'classification' is your target
X = df['dm'].values.reshape(-1,1)
y = df['class']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Creating the Logistic Regression model
classifier = LogisticRegression(random_state = 0)

# Training the model
classifier.fit(X_train, y_train)

# Predicting the test set results
y_pred = classifier.predict(X_test)

# Evaluating the model
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix: \n", cm)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)


Confusion Matrix: 
 [[13  0]
 [13 15]]
Accuracy:  0.6829268292682927


# Random Forest

This script is about the Random Forest algorithm to a dataset for classification purposes, using the RandomForestClassifier class from the sklearn.ensemble module. Here are the key steps:

RandomForestClassifier(n_estimators=100, random_state=42): This creates an instance of the RandomForestClassifier class with 100 trees in the forest (n_estimators=100) and a specified random state for reproducibility (random_state=42).

rf.fit(X_train, y_train): This fits the Random Forest model to the training data. The model learns the relationship between the features (X_train) and the target (y_train) based on an ensemble of decision trees.

rf.predict(X_test): This uses the fitted model to make predictions on the test data.

accuracy_score(y_test, y_pred_rf): This computes the accuracy of the model by comparing the predicted values to the actual values.

The Random Forest algorithm is a type of ensemble learning method, where multiple learning algorithms are used to obtain better predictive performance. In the case of Random Forest, it builds multiple decision trees and merges them together to get a more accurate and stable prediction.

**Hemoglobin A1c (hemo)**

In [ ]:
# Import the necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# Load your data into a pandas DataFrame
# df = pd.read_csv('your_data.csv')

# Assume that 'df' is your DataFrame, 'hemo' is the feature column, and 'classification' is the target column

# Handle missing values in 'hemo' column by filling with the mean
df['hemo'] = df['hemo'].fillna(df['hemo'].mean())

X = df[['hemo']]  # feature
y = df['class']  # target

# Split the dataset into 70% training data and 30% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Create an instance of RandomForestClassifier
clf = RandomForestClassifier(random_state=0)

# Train the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Print the accuracy of the model
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 0.8852459016393442


**Hypertension (htn)**

In [ ]:
# Import the necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Load your data into a pandas DataFrame
# df = pd.read_csv('your_data.csv')

# Assume that 'df' is your DataFrame, 'htn' is the feature column, and 'classification' is the target column
# Also assuming that 'classification' is a binary variable

# Handle missing values in 'htn' column by filling with the mode
df['htn'] = df['htn'].fillna(df['htn'].mode()[0])

# Encode 'htn' column to numerical values if it's categorical
le = LabelEncoder()
df['htn'] = le.fit_transform(df['htn'])

X = df[['htn']]  # feature
y = df['class']  # target

# Split the dataset into 70% training data and 30% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Create an instance of RandomForestClassifier
clf = RandomForestClassifier(random_state=0)

# Train the model
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Print the accuracy of the model
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 0.7377049180327869


**Diabetes Mellitus (dm)**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

# Handling NaN values in 'dm' column
df['dm'] = df['dm'].fillna(df['dm'].mean())

# Selecting 'dm' as the feature and 'class' as the target
X = df['dm'].values.reshape(-1,1)
y = df['class'].values

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Initializing the Random Forest Classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)

# Training the model
rf_clf.fit(X_train, y_train)

# Predicting the test set results
y_pred = rf_clf.predict(X_test)

# Printing the confusion matrix and accuracy score
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))



[[15  0]
 [15 21]]
0.7058823529411765
